In [1]:
%load_ext autoreload
%autoreload 2

In [ ]:
import torch
import torchaudio
import yaml
import pickle

from collections import namedtuple
from pathlib import Path
from snac import SNAC
from train_tokenizer import VQVAEModule
import imageio
from src.dataset import Dataset
from src.tokenizer_module import TokenizerModule
from utils.tokenizer_utils import prepare_features, load_fsq_vae, repackage_output
import matplotlib.pyplot as plt

In [2]:
from src import TokenizerModule

In [ ]:
ds = Dataset("dataset", split="eval", val_split=0.1, seed=2, compute_stats=False)

In [ ]:
sample = ds[134] 

pickle_path = sample['metadata']['pickle_path']
vid_id = pickle_path.split("/")[-1].split(".")[0]
vid_path = f"dataset/train/{vid_id}.mp4"

# Read the first frame from the video and display it
frame = imageio.get_reader(vid_path).get_data(0)
plt.imshow(frame)
plt.show()

In [3]:
# tokenizer = TokenizerModule(tokenizer_config="configs/tokenizer_config.yaml")
tokenizer = TokenizerModule.from_pretrained("InternalCan/tokenizer_module")

Loading precomputed statistics from dataset/stats_all.pkl
Loaded feature-wise statistics successfully
Loaded 2045 test samples
['exp']
FSQ Config:
Levels: 4
Output Emb Width: 4
Num Quantizers: 1
Using FSQ
Codebook size: 256
['exp']
FSQ Config:
Levels: 4
Output Emb Width: 4
Num Quantizers: 1
Using FSQ
Codebook size: 256
['c_eyes_lst', 'c_lip_lst', 'kp', 't', 'x_s']
FSQ Config:
Levels: 4
Output Emb Width: 4
Num Quantizers: 1
Using FSQ
Codebook size: 256
['R', 'scale']
FSQ Config:
Levels: 4
Output Emb Width: 4
Num Quantizers: 1
Using FSQ
Codebook size: 256


In [22]:
features = tokenizer.pickle_to_features("dataset/pickles/WQvT1_tQDhg_22.pkl")

In [ ]:
features.shape

In [24]:
indices = tokenizer.features_to_codes(features)

In [ ]:
indices.shape

In [ ]:
reconstr_feats = tokenizer.codes_to_features(indices)

In [ ]:
reconstr_feats.shape

In [ ]:
output = tokenizer.features_to_pickle(sample, reconstr_feats, pickle_path=pickle_path)

In [16]:
new_path = "dataset/pickles/WQvT1_tQDhg_22_reconstructed.pkl"

In [17]:
# # Replace reconstructed features with original ones for a specific range
# feat = 'exp'

# feat_range = (0, 21)

# denormalized_sample = ds.denormalize_features(sample[feat], feat)

# for i in range(output['n_frames']):
#     output['motion'][i][feat][:, feat_range[0]:feat_range[1]] = 0

In [ ]:
!python inference.py -d {new_path} -s assets/examples/source/reconstructed.png

In [ ]:
# tokenizer.save_pretrained("tokenizer_module")
# tokenizer.push_to_hub("InternalCan/tokenizer_module")